# core

> Utility class for working with TEI files.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import urllib.request
import os
from typing import Dict
import os
from xml.sax import parse
from xml.sax.saxutils import XMLGenerator
import xml.etree.ElementTree as ET
import glob

In [ ]:
#| export
class CycleFile(object):

    def __init__(self, filename):
        self.basename, self.ext = os.path.splitext(filename)
        self.index = 0
        self.open_next_file()

    def open_next_file(self):
        self.index += 1
        self.file = open(self.name(), 'wb')

    def name(self):
        return '%s-p%s%s' % (self.basename, str(self.index).zfill(4), self.ext)

    def cycle(self):
        self.file.close()
        self.open_next_file()

    def write(self, str):
        self.file.write(str)

    def close(self):
        self.file.close()

#| export
class XMLBreaker(XMLGenerator):
    
    def __init__(self, split_element, break_after=1000, split_attr=None, split_value=None, out=None, *args, **kwargs):
        super().__init__(out, *args, **kwargs)
        self.out_file = out
        self.split_element = split_element
        self.split_attr = split_attr
        self.split_value = split_value
        self.break_after = break_after
        self.context = []
        self.count = 0

    def startElement(self, name, attrs):
        super().startElement(name, attrs)
        self.context.append((name, attrs))

        # Check if this element is the one to split on
        if name == self.split_element:
            if self.split_attr is None or attrs.get(self.split_attr) == self.split_value:
                self.count += 1
                if self.count >= self.break_after:
                    self.split()

    def endElement(self, name):
        super().endElement(name)
        self.context.pop()

    def split(self):
        # Reset the counter
        self.count = 0
        # Close elements up to the split point
        for element in reversed(self.context):
            self.out_file.write("\n".encode('utf-8'))
            super().endElement(element[0])
        # Cycle to the next file
        self.out_file.cycle()
        # Reopen the XML context in the new file
        super().startDocument()
        for element in self.context:
            super().startElement(*element)

def convert_xml_encoding(input_file, output_file):
    # Parse the XML file
    tree = ET.parse(input_file)

    # Define the new XML declaration
    new_declaration = '<?xml version="1.0" encoding="UTF-8"?>\n'
    
    # Write the new declaration followed by the XML content
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(new_declaration)
        tree.write(file, encoding='unicode')

In [ ]:
#| export
class TeiUtils:
    """ Utility class for working with TEI files. """
    def __init__(self):
        self.tag_counts = defaultdict(int)
        self.df = None
        self.df_tag = None

    def download(self, url: str, path: str) -> None:
        """ Download a file from a specified URL to a local path.
        
        Args:
            url: The URL from which to download the file.
            path: The local file path to save the downloaded file.
        """
        os.makedirs(os.path.dirname(path), exist_ok=True)
        urllib.request.urlretrieve(url, path)

    def get_tag_freq(self, path: str) -> None:
        """ Read an XML file from a specified path and count the frequency of each tag.
        
        The frequencies are stored in an attribute `tag_counts`.
        A sorted DataFrame of tags and counts is stored in `df` and `df_tag`.
        
        Args:
            path: The file path of the XML file to parse.
        """
        with open(path, "r") as f:
            xml = f.read()

        soup = BeautifulSoup(xml, 'lxml-xml')

        for tag in soup.find_all(True):
            self.tag_counts[tag.name] += 1

        self.df = pd.DataFrame(list(self.tag_counts.items()), columns=["Tag", "Count"])
        self.df.sort_values(by="Count", ascending=False, inplace=True)

        self.df_tag = self.df.sort_values(by="Tag", ascending=True)

    def get_javascript(self) -> None:
        """ Generate JavaScript code to check checkboxes in the TEI tag list. """

        tags = []

        for i, row in self.df_tag.iterrows():
            tags.append(f"\"{row['Tag']}\"")

        javascript_code = f"""
function checkCheckboxesWithTextValues(textValues) {{
    // 存在しなかった要素名を格納する配列
    let notFound = [];

    // 指定されたテキスト値のリストをループ処理
    textValues.forEach(function(textToMatch) {{
        // テキストに一致する .mdc-list-item__primary-text 要素を取得
        let found = false;
        document.querySelectorAll('.mdc-list-item__primary-text').forEach(function(item) {{
            if (item.textContent.trim() === textToMatch) {{
                found = true;
                let checkbox = item.closest('.mdc-list-item').querySelector('.mdc-checkbox__native-control');
                if (checkbox) {{
                    checkbox.checked = true;
                }}
            }}
        }});

        // 要素が見つからなければ notFound 配列に追加
        if (!found) {{
            notFound.push(textToMatch);
        }}
    }});

    // 存在しなかった要素名を返す
    return notFound;
}}

// 指定したいテキスト値のリスト
const itemsToCheck = [{", ".join(tags)}];

// チェックしたい項目のリストを関数に渡し、存在しなかった項目を取得
const itemsNotFound = checkCheckboxesWithTextValues(itemsToCheck);

// 存在しなかった項目をコンソールに出力
if (itemsNotFound.length > 0) {{
    console.log('These items were not found:', itemsNotFound);
}} else {{
    console.log('All items were found and checked.');
}}
"""

        return javascript_code
    
    def split_xml_file(self, input_file, output_basename, split_element, break_after=1, split_attr=None, split_value=None):
        """Splits an XML file into multiple files based on the provided element and attributes.

        Args:  
            input_file (str): The path to the input XML file.  
            output_basename (str): The base name for the output files.  
            split_element (str): The name of the element to split on.  
            break_after (int): The number of occurrences of the element after which to split.  
            split_attr (str): Optional. The attribute name to further refine the split condition.  
            split_value (str): Optional. The attribute value to further refine the split condition.  
        """

        # 
        os.makedirs(os.path.dirname(output_basename), exist_ok=True)

        # Create a CycleFile instance for the output.
        out = CycleFile(output_basename)
        
        # Create an XMLBreaker instance, passing the CycleFile instance as the output.
        parser = XMLBreaker(split_element=split_element, split_attr=split_attr, split_value=split_value, break_after=break_after, out=out)
        
        # Parse the input file with the configured parser.
        parse(input_file, parser)
        
        # Close the current file.
        out.close()

        # Convert the encoding of the output files to UTF-8.
        basename = os.path.basename(output_basename).split('.')[0]
        dirpath = os.path.dirname(output_basename)
        path = os.path.join(dirpath, f'{basename}-p*.xml')
        for file in glob.glob(path):
            convert_xml_encoding(file, file)

In [ ]:
show_doc(TeiUtils.download)

---

[source](https://github.com/nakamura196/gdb-utils/blob/main/gdb_utils/core.py#L93){target="_blank" style="float:right; font-size:smaller"}

### TeiUtils.download

>      TeiUtils.download (url:str, path:str)

Download a file from a specified URL to a local path.

Args:
    url: The URL from which to download the file.
    path: The local file path to save the downloaded file.

In [ ]:
show_doc(TeiUtils.get_tag_freq)

---

[source](https://github.com/nakamura196/gdb-utils/blob/main/gdb_utils/core.py#L103){target="_blank" style="float:right; font-size:smaller"}

### TeiUtils.get_tag_freq

>      TeiUtils.get_tag_freq (path:str)

Read an XML file from a specified path and count the frequency of each tag.

The frequencies are stored in an attribute `tag_counts`.
A sorted DataFrame of tags and counts is stored in `df` and `df_tag`.

Args:
    path: The file path of the XML file to parse.

In [ ]:
show_doc(TeiUtils.get_javascript)

---

[source](https://github.com/nakamura196/gdb-utils/blob/main/gdb_utils/core.py#L125){target="_blank" style="float:right; font-size:smaller"}

### TeiUtils.get_javascript

>      TeiUtils.get_javascript ()

Generate JavaScript code to check checkboxes in the TEI tag list.

In [ ]:
show_doc(TeiUtils.split_xml_file)

---

[source](https://github.com/nakamura196/gdb-utils/blob/main/gdb_utils/core.py#L178){target="_blank" style="float:right; font-size:smaller"}

### TeiUtils.split_xml_file

>      TeiUtils.split_xml_file (input_file, output_basename, split_element,
>                               break_after=1, split_attr=None,
>                               split_value=None)

Splits an XML file into multiple files based on the provided element and attributes.

Args:  
    input_file (str): The path to the input XML file.  
    output_basename (str): The base name for the output files.  
    split_element (str): The name of the element to split on.  
    break_after (int): The number of occurrences of the element after which to split.  
    split_attr (str): Optional. The attribute name to further refine the split condition.  
    split_value (str): Optional. The attribute value to further refine the split condition.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()